In [1]:
# Day 7 - part2 challenge for AOC2019
# https://adventofcode.com/2019/day/7

In [2]:
# this object is where the Intcode program lives and some useful methods
# to fill it, clear it, read from it, write to it
class Computer:
    global set_trace
    global inputs
    
    def __init__(self, size, stage):
        self.size = size  # must specify the memory size
        self.stage = stage
        self.storage = [None] * size  # initialize with None
     
    # dump out the contents between 2 positions (defaults all the memory)
    def dump(self,start=0,end=-1):
        if(end == -1):
            end = self.size
        for i in range(start, end):
            print("{}: {}".format(i,self.storage[i]))

    # fill the memory with a sequence, starting at a certain position - which defaults to 0
    def fill(self, codelist, start_pos = 0):
        i = start_pos
        for j in codelist:
            self.storage[i] = j
            i+=1
    
    # fill (write to) memory with a sequence, allowing direct or reference starting location
    def mode_fill(self, s, mem_location, mode):
        # mem_location: where to begin fill or to reference to begin
        # mode (0 - position, 1 - immediate)
        if(set_trace): import pdb; pdb.set_trace()
        if(mode == '0'):
            i = self.storage[mem_location]
            for j in s:
                self.storage[i] = j
                i+=1
        elif(mode == '1'):
            i = mem_location
            for j in s:
                self.storage[i] = j
                i+=1
                
    # get data from input_buffer for this stage
    # if it has 2 values, then it hasn't read in the amp_setting
    # that state will return the last value and remove it
    def getfrominput(self):
        if(len(inputs[self.stage].contents) > 1):
            # pop a value off and return it
            return(inputs[self.stage].contents.pop())
        else:
            # return just that last value but leave it
            return(inputs[self.stage].contents[0])

    # put data to output_buffer for this stage (which is next stage's input)
    # put this output value into position [0] of that buffer (position [1] will have possible amp_setting)
    def puttooutput(self, val):
        inputs[next_stage(self.stage,5)].contents[0] = val
        return

    # read the contents from just one position
    def read(self, pos):
        return self.storage[pos]
   
    # return the value based on parameter mode (position or immediate)
    def mode_read(self, pos, mode):
        if(set_trace): import pdb; pdb.set_trace()
        # mode: 0 - position, 1 - immediate
        if(mode == '0'):
            return int(self.storage[self.storage[pos]])
        elif(mode == '1'):
            return int(self.storage[pos])

    # clear the memory
    def clear(self):
        self.storage = [None] * self.size

    # this function will execute a single sequence (operator + subsequent operands and output location)
    def exec_seq(self, ctr):
        global set_trace
        # ctr - where to start
        # m - the memory object to work from
        # the first in the sequence is the 'enhnanced opcode' (includes 3 parameter modes in most significant position)
        # zero-pad to 5 - most significant 0 is implied
        enhanced_opcode = str(self.read(ctr)).zfill(5)
        # first 3 characters are the parameter modes
        pmodes = enhanced_opcode[:3]
        # last 2 characters are the opcode
        opcode = enhanced_opcode[3:]
        if(set_trace): import pdb; pdb.set_trace()
        # if it is 99, we're done
        if(opcode == '99'):
            prog_status[self.stage] = "STOP"
            return("STOP")
        # this is the "add" operator
        elif(opcode == "01"):
            if(set_trace): import pdb; pdb.set_trace()
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(set_trace): import pdb; pdb.set_trace()
            calcval = operand1 + operand2
        # this is the "multiply" operator
        elif(opcode == "02"):
            if(set_trace): import pdb; pdb.set_trace()
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            calcval = operand1 * operand2
        # this is the 'store from input' operator
        elif(opcode == "03"):
            if(set_trace): import pdb; pdb.set_trace()
            # get the input
            # val = getfrominput()
            # the input buffer is the one for this amp stage
            val = self.getfrominput()
            # store it in address provided by next code
            self.mode_fill([val], ctr+1, pmodes[2])
            # return next prog counter
            return(ctr+2)
        # this is the 'send to output' operator
        elif(opcode == "04"):
            if(set_trace): import pdb; pdb.set_trace()
            # read the val (based on mode) specified from next code and send to output
            self.puttooutput(self.mode_read(ctr+1, pmodes[2]))
            prog_status[self.stage] = "OUTPUT"
            return(ctr+2)
        # this is 'jump if true'
        elif(opcode == "05"):
            if(self.mode_read(ctr+1, pmodes[2])):
                return(self.mode_read(ctr+2, pmodes[1]))
            else:
                return(ctr+3)
        # this is 'jump if false'
        elif(opcode == "06"):
            if(self.mode_read(ctr+1, pmodes[2])):
                return(ctr+3)
            else:
                return(self.mode_read(ctr+2, pmodes[1]))
        # this is 'less than' operator
        elif(opcode == "07"):
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(operand1 < operand2):
                storeval = 1
            else:
                storeval = 0
            self.mode_fill([storeval], ctr+3, pmodes[0])
            return(ctr+4)
        # this is the 'eqals to' operator
        elif(opcode == "08"):
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(operand1 == operand2):
                storeval = 1
            else:
                storeval = 0
            self.mode_fill([storeval], ctr+3, pmodes[0])
            return(ctr+4)        
        # if here, then the opcode is not known
        else:
            return("BADOPCODE")
        # fill the output destination with value depending on mode
        if(set_trace): import pdb; pdb.set_trace()
        self.mode_fill([calcval], ctr+3, pmodes[0])
        # and return the next postion in the memory
        return(ctr+4)

In [3]:
# this returns the next stage #, and 0 if it would be max (feedback thing)
def next_stage(i, max):
    n = i+1
    if(n == max):
        return(0)
    else:
        return(n)

In [4]:
# this will turn on debugging and set breakpoints
set_trace = False

In [5]:
class Amplifier:
    def __init__(self, size, stage=0):
        self.phase_setting = None
        self.stage = stage
        self.computer = Computer(size, stage=stage)

In [6]:
class Input_buffer:
    def __init__(self):
        self.contents = [None, None]

In [7]:
# this function returns a list that contains only the unique elements of the input list
def uniquify(seq):
   # Not order preserving    
   retset = set(seq)
   return list(retset)

In [8]:
# this function fills the amp_settings array with only unique elements from the input list
def fill_amp_settings(obj_list):
    global amp_settings
    amp_settings = []
    for i in obj_list:
        for j in obj_list:
            for k in obj_list:
                for l in obj_list:
                    for m in obj_list:
                        pot_list = [i,j,k,l,m]
                        if(len(uniquify(pot_list)) == len(obj_list)):
                            amp_settings.append(pot_list)

In [9]:
# input buffers - read from by each amp stage
inputs = [Input_buffer() for _ in range(5)]

In [10]:
# this is the initial input (read into the input buffer before execution)
inputs[0].contents[0] = 0

# the amp 'program'
seq = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
# seq = [3,8,1001,8,10,8,105,1,0,0,21,42,67,84,109,126,207,288,369,450,99999,3,9,102,4,9,9,1001,9,4,9,102,2,9,9,101,2,9,9,4,9,99,3,9,1001,9,5,9,1002,9,5,9,1001,9,5,9,1002,9,5,9,101,5,9,9,4,9,99,3,9,101,5,9,9,1002,9,3,9,1001,9,2,9,4,9,99,3,9,1001,9,2,9,102,4,9,9,101,2,9,9,102,4,9,9,1001,9,2,9,4,9,99,3,9,102,2,9,9,101,5,9,9,1002,9,2,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,99]

In [11]:
# fill the amp_settings array with unique combinations of the following symbols
fill_amp_settings(['0','1','2','3','4'])

In [12]:
# a single sequence of amp settings (for testing)
amp_settings = []
amp_settings.append([9,8,7,6,5])

In [14]:
# initialize the input buffers with the amp_settings
for idx,s in enumerate(amp_settings[0]):
    inputs[idx].contents[1] = s
    # print(inputs[idx].contents)

In [15]:
prog_status = [None] * 5
prog_ctr = [None] * 5
amps = [Amplifier(len(seq), stage=i) for i in range(5)]

In [23]:
# initialize each stage
for i in range(5):
    amps[i].computer.fill(seq)
    # prog_ctr[i] = 0
    prog_status[i] = "IDLE"
print(prog_status[0])
print(prog_ctr[0])

IDLE
STOP


In [22]:
while(prog_status[4] != "STOP"):
    for i in range(5):
        prog_status[i] = "RUNNING"
        print("prog_ctr[{}]={}".format(i, prog_ctr[i]))
        prog_ctr[i] = amps[i].computer.exec_seq(prog_ctr[i])
        while(prog_status[i] != "OUTPUT" and prog_status[i] != "STOP"):
            prog_ctr[i] = amps[i].computer.exec_seq(prog_ctr[i])
        prog_status[i] = "IDLE"

print(inputs[0].contents[0])

prog_ctr[0]=STOP


TypeError: list indices must be integers or slices, not str

In [ ]:
# initialize the max
max = 0
# initialize the input to the whole system
prog_inputs = []
outputstring = '0'

for j in range(0,4):
    for a in amp_settings:

        # go through as many stages are as there are settings
        for i in range(0,len(a)):
            # instantiate the amp
            amp = Amplifier(len(seq))
            # load the program
            amp.computer.fill(seq)
            # set the the inputs
            # the input to this stage is the output from the last stage (persisted in outputstring)
            prog_inputs.append(outputstring)
            outputstring = None
            # put this stage's setting onto the input buffer
            prog_inputs.append(a[i])
            # run the program
            # execute the program
            # print(prog_inputs)
            prog_ctr = 0
            prog_status = "RUNNING"
            ret = amp.computer.exec_seq(prog_ctr)
            while(ret != "STOP" and ret != "BADOPCODE"):
                # print(ret)
                prog_ctr = ret
                ret = amp.computer.exec_seq(ret)
            prog_status = ret
            print("PROG STATE AFTER CYCLE: {}, AMP#{}: {}".format(j,i,prog_status))
            if(set_trace): import pdb; pdb.set_trace()
            
            prog_inputs = []

        # print("final output: {}".format(outputstring))
        outnum = int(outputstring)
    if(outnum > max):
        max = outnum
        s = a

In [ ]:
print("settings: {} produces a max thruster output of: {}".format(s,max))